In [1]:
import requests
import json

In [2]:
analysisSettings={}
mappingSettings={}

### Extract movies from tmbd.json

In [4]:
def extract():
    f = open('tmdb.json')
    if f:
        return json.loads(f.read());

### Indexing with Bulk API

In [8]:
def reindex(analysisSettings={}, mappingSettings={}, movieDict={}):
    settings = {
        "number_of_ shards": 1,
        "index": {
            "analysis": analysisSettings,
        }
    }
    if mappingSettings:
        settings['mappings'] = mappingSettings
        
    resp = requests.delete('http://192.168.32.10:9200/tmdb')
    resp = requests.put('http://192.168.32.10:9200/tmdb', data=json.dumps(settings))
    bulkMovies = ""
    for id, movie in movieDict.iteritems():
        addCmd = {"index": {"_index": "tmdb",
                           "_type": "movie",
                           "_id": movie["id"]}}
        bulkMovies += json.dumps(addCmd) + "\n" + json.dumps(movie) + "\n"
    resp = requests.post("http://192.168.32.10:9200/_bulk", data=bulkMovies)

### Pulling data into Elasticsearch

In [9]:
movieDict = extract()
reindex(movieDict=movieDict)

### Search Function

In [11]:
def search(query):
    url = 'http://192.168.32.10:9200/tmdb/movie/_search'
    httpResp = requests.get(url, data=json.dumps(query))
    searchHits = json.loads(httpResp.text)['hits']
    print "Num\tRelevance Score \t\tMovie Title"
    for idx, hit in enumerate(searchHits['hits']):
        print "%s\t%s\t\t%s" % (idx + 1, hit['_score'], hit['_source']['title'])

### Search query

In [14]:
usersSearch = 'william shater patrick stewart'
query = {
    "query": {
        "multi_match": {
            "query": usersSearch,
            "fields": ["overview", "title", "directors.name", "cast.name"],
            "type": "cross_fields"
        }
    }
}

In [15]:
search(query)

Num	Relevance Score 		Movie Title
1	0.4553907		Dark Skies
2	0.3394483		Star Trek: Generations
3	0.33406225		Star Trek: Nemesis
4	0.28889155		Drive Angry
5	0.24757117		The Collector
6	0.24694258		X2: X-Men United
7	0.20350662		Star Trek: Insurrection
8	0.20350662		Excalibur
9	0.20208357		Legion
10	0.20208357		Halo 4: Forward Unto Dawn
